In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [ ]:
#Loading the dataset
data = pd.read_csv("/content/Womens Clothing E-Commerce Reviews.csv")

In [ ]:
#Dropping rows with missing values in the Review Text column
data.dropna(subset=['Review Text'], inplace=True)


In [ ]:
#Mapping ratings to sentiments (e.g., ratings 4 and 5 as positive, rating 3 as neutral, ratings 1 and 2 as negative)
data['Sentiment'] = data['Rating'].apply(lambda x: 'positive' if x > 3 else ('neutral' if x == 3 else 'negative'))

In [ ]:
#Tokenization of the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Review Text'])
X = tokenizer.texts_to_sequences(data['Review Text'])
X = pad_sequences(X, maxlen=100)  # Limit reviews to 100 words

In [ ]:
#Encoding the sentiment labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['Sentiment'])

In [ ]:
#Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=100))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=3, activation='softmax'))

In [ ]:
#Compiling the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
#Training the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
255/255 [==============================] - 113s 424ms/step - loss: 0.5367 - accuracy: 0.7951 - val_loss: 0.4859 - val_accuracy: 0.7930
Epoch 2/5
255/255 [==============================] - 93s 364ms/step - loss: 0.3832 - accuracy: 0.8334 - val_loss: 0.5112 - val_accuracy: 0.7964
Epoch 3/5
255/255 [==============================] - 89s 351ms/step - loss: 0.3317 - accuracy: 0.8598 - val_loss: 0.5058 - val_accuracy: 0.7820
Epoch 4/5
255/255 [==============================] - 87s 340ms/step - loss: 0.2780 - accuracy: 0.8863 - val_loss: 0.5411 - val_accuracy: 0.7897
Epoch 5/5
255/255 [==============================] - 87s 340ms/step - loss: 0.2300 - accuracy: 0.9101 - val_loss: 0.6075 - val_accuracy: 0.7936


In [ ]:
#Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

142/142 [==============================] - 5s 32ms/step - loss: 0.5626 - accuracy: 0.8053
Test Loss: 0.5625892281532288
Test Accuracy: 0.8052549958229065


In [ ]:
#Perform sentiment analysis on new input
def predict_sentiment(review_text):
    #Tokenize and pad the input text
    input_sequence = tokenizer.texts_to_sequences([review_text])
    padded_sequence = pad_sequences(input_sequence, maxlen=100)
    #Predict the sentiment
    sentiment_probs = model.predict(padded_sequence)[0]
    sentiment_label = label_encoder.classes_[sentiment_probs.argmax()]
    return sentiment_label

In [ ]:
#sample prediction
user_review = "This dress exceeded my expectations! The fabric is luxurious and feels amazing against the skin."
predicted_sentiment = predict_sentiment(user_review)
print(f"The review by user is: '{user_review}'")
print(f"The review is {predicted_sentiment}.")

1/1 [==============================] - 0s 40ms/step
The review by user is: 'This dress exceeded my expectations! The fabric is luxurious and feels amazing against the skin.'
The review is positive.
